In [1]:
import helix
from helix.client import Query
from helix.types import Payload
from typing import List
from instance import Instance

## Check Configs

In [2]:
import json
import os
from pathlib import Path

CONFIG_PATH = "helixdb-cfg"
CONFIG_FILE = "config.hx.json"
QUERIES_FILE = "queries.hx"
SCHEMA_FILE = "schema.hx"
DEFAULT_CONFIG = {
    "vector_config": {
        "m": 16,
        "ef_construction": 128,
        "ef_search": 768
    },
    "graph_config": {
        "secondary_indices": []
    },
    "db_max_size_gb": 10
}

helix_dir = Path(os.path.dirname(os.path.curdir)).resolve()
os.makedirs(os.path.join(helix_dir, CONFIG_PATH), exist_ok=True)

if not Path(os.path.join(helix_dir, CONFIG_PATH, CONFIG_FILE)).exists():
    configs = DEFAULT_CONFIG
    path = os.path.join(helix_dir, CONFIG_PATH, CONFIG_FILE)
    with open(path, "w") as f:
        json.dump(configs, f, indent=4)
    print("Config file created")
else:
    path = os.path.join(helix_dir, CONFIG_PATH, CONFIG_FILE)
    with open(path, "r") as f:
        configs = json.load(f)
    print("Config file loaded")

Config file loaded


## Create Schema

In [3]:
def create_node(node_type:str, properties:dict = {}) -> str:
    output = ""
    output += "N::" + node_type + " {\n"
    for key, value in properties.items():
        output += "    " + str(key) + ": " + str(value) + "\n"
    output += "}\n"
    return output

print(create_node("User", {"name": "String", "age": "U32"}))

N::User {
    name: String
    age: U32
}



In [4]:
def create_edge(edge_type:str, from_node:str, to_node:str, properties:dict = {}) -> str:
    output = ""
    output += "E::" + edge_type + " {\n"
    output += "    From: " + from_node + "\n"
    output += "    To: " + to_node + "\n"
    output += "    Properties: {\n"
    for key, value in properties.items():
        output += "        " + str(key) + ": " + str(value) + "\n"
    output += "    }\n"
    output += "}\n"
    return output

print(create_edge("Follows", "User", "User", {"since": "I32"}))

E::Follows {
    From: User
    To: User
    Properties: {
        since: I32
    }
}



In [5]:
def create_vector(vector_type:str, properties:dict = {}) -> str:
    output = ""
    output += "V::" + vector_type + " {\n"
    for key, value in properties.items():
        output += "    " + str(key) + ": " + str(value) + "\n"
    output += "}\n"
    return output

print(create_vector("User", {"name": "String", "age": "U32"}))

V::User {
    name: String
    age: U32
}



## Helix CLI

### Deploy

In [6]:
import subprocess
import re
import pexpect

# Run command in background
deploy_db = subprocess.Popen(
    ['helix', 'deploy', '--path', 'helixdb-cfg', '--port', '6969'],
    stdout=subprocess.PIPE,
    text=True
)

output = []

# Read output line by line
for line in deploy_db.stdout:
    process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    line = process_line.sub('', line)
    output.append(line.strip())
    print(line.strip())

# Wait for process to complete
deploy_db.wait()

if "error" in "\n".join(output).lower():
    raise Exception(f"Failed to deploy Helix instance")

instance_id = [out for out in output if out.startswith("Instance ID:")][0].split("Instance ID: ")[1].split(" (running)")[0]
# print('\n'.join(output))
print(f"Instance ID: {instance_id}")

Successfully compiled 2 query files
⢺ Building Helix

Successfully transpiled queries
Successfully wrote queries file


Successfully built Helix
⢹ Starting Helix instance

Instance ID: 9198b99e-c28d-4bb1-ad27-1c739f78e8f2 (running)
└── Label:
└── Port: 6969
└── Available endpoints:
└── /get_users
└── /get_followed_users_posts
└── /get_followed_users
└── /create_post
└── /get_posts
└── /get_posts_by_user
└── /create_user
└── /create_follow
Instance ID: 9198b99e-c28d-4bb1-ad27-1c739f78e8f2


Successfully started Helix instance


### Redeploy

In [8]:
redeploy_db = subprocess.Popen(
    ['helix', 'redeploy', '--path', 'helixdb-cfg', instance_id],
    stdout=subprocess.PIPE,
    text=True
)

output = []

# Read output line by line
for line in redeploy_db.stdout:
    process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    line = process_line.sub('', line)
    output.append(line.strip())
    print(line.strip())

# Wait for process to complete
redeploy_db.wait()

if "error" in "\n".join(output).lower():
    print('\n'.join(output))
    raise Exception(f"Failed to redeploy Helix instance")

# instance_id = [out for out in output if out.startswith("Instance ID:")][0].split("Instance ID: ")[1].split(" (running)")[0]
# print('\n'.join(output))
# print(f"Instance ID: {instance_id}")

Successfully compiled 2 query files
⢺ Building Helix

Helix instance found!
Successfully wrote queries file


⢹ Building Helix

Instance ID: 9198b99e-c28d-4bb1-ad27-1c739f78e8f2 (running)
└── Label:
└── Port: 6969
└── Available endpoints:
└── /get_followed_users
└── /get_posts
└── /get_posts_by_user
└── /get_users
└── /create_follow
└── /create_post
└── /get_followed_users_posts
└── /create_user


Successfully built Helix
Successfully started Helix instance


### Stop

In [9]:
stop_db = subprocess.Popen(
    ['helix', 'stop', instance_id],
    stdout=subprocess.PIPE,
    text=True
)

output = []

for line in stop_db.stdout:
    process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    line = process_line.sub('', line)
    output.append(line.strip())

stop_db.wait()

print('\n'.join(output))

Stopped instance 9198b99e-c28d-4bb1-ad27-1c739f78e8f2


### Start

In [10]:
start_db = subprocess.Popen(
    ['helix', 'start', instance_id],
    stdout=subprocess.PIPE,
    text=True
)

output = []

# Read output line by line
for line in start_db.stdout:
    process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    line = process_line.sub('', line)
    output.append(line.strip())

# Wait for process to complete
start_db.wait()

print('\n'.join(output))

Instance ID: 9198b99e-c28d-4bb1-ad27-1c739f78e8f2 (running)
└── Label:
└── Port: 6969
└── Available endpoints:
└── /get_followed_users
└── /get_posts
└── /get_posts_by_user
└── /get_users
└── /create_follow
└── /create_post
└── /get_followed_users_posts
└── /create_user


Successfully started Helix instance


### Delete

In [11]:
delete_db = pexpect.spawn(
    ' '.join(['helix', 'delete', instance_id]),
)

delete_db.expect("Are you sure you want to delete", timeout=10)
delete_db.sendline("y")
delete_db.expect(pexpect.EOF, timeout=10)

del_out = delete_db.before.decode('utf-8').split('\n')

for line in del_out:
    process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    line = process_line.sub('', line)
    print(line)

 the instance and its data? (y/n): y
Deleted Helix instance
Deleted Helix instance data



### View Instances

In [12]:
status_db = subprocess.Popen(
    ['helix', 'instances'],
    stdout=subprocess.PIPE,
    text=True
)

output = []

# Read output line by line
for line in status_db.stdout:
    process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    line = process_line.sub('', line)
    output.append(line.strip())

# Wait for process to complete
status_db.wait()

print('\n'.join(output))

No running Helix instances


### Instance class

In [13]:
# import subprocess
# import re
# import pexpect
# from helix.types import GHELIX, RHELIX

# class Instance:
#     def __init__(self, config_path=None, port=None, verbose=True):
#         self.config_path = config_path
#         self.port = port
#         self.instance_id = None

#         self.verbose = verbose
#         self.process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

#     def deploy(self):
#         if self.verbose: print(f"{GHELIX} Deploying Helix instance")
#         cmd = ['helix', 'deploy']
#         if self.config_path:
#             cmd.extend(['--path', self.config_path])
#         if self.port:
#             cmd.extend(['--port', str(self.port)])

#         process = subprocess.Popen(
#             cmd,
#             stdout=subprocess.PIPE,
#             text=True
#         )

#         output = []

#         for line in process.stdout:
#             line = self.process_line.sub('', line)
#             output.append(line.strip())
#             if self.verbose: print(line.strip())

#         process.wait()

#         if "error" in "\n".join(output).lower():
#             raise Exception(f"{RHELIX} Failed to deploy Helix instance")

#         self.instance_id = [out for out in output if out.startswith("Instance ID:")][0].split("Instance ID: ")[1].split(" (running)")[0]

#         if self.verbose: print(f"{GHELIX} Deployed Helix instance: {self.instance_id}")

#         return output
    
#     def redeploy(self):
#         if self.verbose: print(f"{GHELIX} Redeploying Helix instance: {self.instance_id}")
#         cmd = ['helix', 'redeploy', '--path', self.config_path, self.instance_id]
#         process = subprocess.Popen(
#             cmd,
#             stdout=subprocess.PIPE,
#             text=True
#         )

#         output = []

#         for line in process.stdout:
#             line = self.process_line.sub('', line)
#             output.append(line.strip())
#             if self.verbose: print(line.strip())

#         process.wait()

#         if "error" in "\n".join(output).lower():
#             raise Exception(f"Failed to redeploy Helix instance")

#         return output

#     def start(self):
#         if self.verbose: print(f"{GHELIX} Starting Helix instance: {self.instance_id}")
#         cmd = ['helix', 'start', self.instance_id]
#         process = subprocess.Popen(
#             cmd,
#             stdout=subprocess.PIPE,
#             text=True
#         )

#         output = []

#         for line in process.stdout:
#             line = self.process_line.sub('', line)
#             output.append(line.strip())
#             if self.verbose: print(line.strip())

#         process.wait()

#         if "error" in "\n".join(output).lower():
#             raise Exception(f"{RHELIX} Failed to start Helix instance")

#         return output

#     def stop(self):
#         if self.verbose: print(f"{GHELIX} Stopping Helix instance: {self.instance_id}")
#         process = subprocess.Popen(
#             ['helix', 'stop', self.instance_id],
#             stdout=subprocess.PIPE,
#             text=True
#         )

#         output = []

#         for line in process.stdout:
#             line = self.process_line.sub('', line)
#             output.append(line.strip())
#             if self.verbose: print(line.strip())

#         process.wait()

#         if "error" in "\n".join(output).lower():
#             raise Exception(f"{RHELIX} Failed to stop Helix instance")

#         return output

#     def delete(self):
#         if self.verbose: print(f"{GHELIX} Deleting Helix instance: {self.instance_id}")
#         process = pexpect.spawn(
#             ' '.join(['helix', 'delete', self.instance_id]),
#         )

#         process.expect("Are you sure you want to delete", timeout=10)
#         process.sendline("y")
#         process.expect(pexpect.EOF, timeout=10)

#         output = process.before.decode('utf-8').split('\n')

#         for line in output[1:]:
#             line = self.process_line.sub('', line)
#             if self.verbose: print(line)

#         if "error" in "\n".join(output).lower():
#             raise Exception(f"{RHELIX} Failed to delete Helix instance")

#         return output

#     def status(self):
#         if self.verbose: print(f"{GHELIX} Helix instance status:")
#         cmd = ['helix', 'instances']
#         process = subprocess.Popen(
#             cmd,
#             stdout=subprocess.PIPE,
#             text=True
#         )

#         output = []

#         for line in process.stdout:
#             line = self.process_line.sub('', line)
#             output.append(line.strip())
#             if self.verbose: print(line.strip())

#         process.wait()

#         if "error" in "\n".join(output).lower():
#             raise Exception(f"{RHELIX} Failed to get Helix instance status")

#         return output

In [14]:
# helix_instance = Instance("helixdb-cfg", "6969")
# print("-" * 50 + '\n')
# helix_instance.deploy()
# print("-" * 50 + '\n')
# helix_instance.status()
# print("-" * 50 + '\n')
# helix_instance.stop()
# print("-" * 50 + '\n')
# helix_instance.status()
# print("-" * 50 + '\n')
# helix_instance.start()
# print("-" * 50 + '\n')
# helix_instance.status()
# print("-" * 50 + '\n')
# helix_instance.delete()
# print("-" * 50 + '\n')
# helix_instance.status()
# print("-" * 50 + '\n')

In [1]:
from instance import Instance
helix_instance = Instance("helixdb-cfg", "6969")
print("-" * 50 + '\n')
helix_instance.deploy()
print("-" * 50 + '\n')
helix_instance.status()
print("-" * 50 + '\n')
helix_instance.stop()
print("-" * 50 + '\n')
helix_instance.status()
print("-" * 50 + '\n')
helix_instance.start()
print("-" * 50 + '\n')
helix_instance.status()
print("-" * 50 + '\n')
helix_instance.delete()
print("-" * 50 + '\n')
helix_instance.status()
print("-" * 50 + '\n')

--------------------------------------------------

[HELIX] Deploying Helix instance
Successfully transpiled queries
Successfully wrote queries file


Successfully compiled 2 query files
Successfully built Helix
Successfully started Helix instance
Successfully started Helix instance


Instance ID: b30ec204-086e-4a26-a9e2-0f4ccd255e45 (running)
└── Label:
└── Port: 6969
└── Available endpoints:
└── /create_follow
└── /get_users
└── /get_followed_users_posts
└── /get_posts_by_user
└── /create_post
└── /get_posts
└── /get_followed_users
└── /create_user
[HELIX] Deployed Helix instance: b30ec204-086e-4a26-a9e2-0f4ccd255e45
--------------------------------------------------

[HELIX] Helix instance status:
Instance ID: b30ec204-086e-4a26-a9e2-0f4ccd255e45 (running)
└── Label:
└── Port: 6969
└── Available endpoints:
└── /create_follow
└── /get_users
└── /get_followed_users_posts
└── /get_posts_by_user
└── /create_post
└── /get_posts
└── /get_followed_users
└── /create_user

--------------------------------------------------

[HELIX] Stopping Helix instance: b30ec204-086e-4a26-a9e2-0f4ccd255e45
Stopped instance b30ec204-086e-4a26-a9e2-0f4ccd255e45
--------------------------------------------------

[HELIX] Helix instance status:
Instance ID: b30ec204-086e-4a26-a9e2-0f4ccd25

## Create RAG Query